In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

retaildata = spark.read.csv(
    path = "2010-12-01.csv",
    sep = ",",
    header=True,
    quote='"',
    schema="InvoiceNo INT, StockCode STRING, Description STRING,Quantity INT, InvoiceDate TIMESTAMP, UnitPrice FLOAT, CustomerID FLOAT, Country STRING")

retaildata.show()
retaildata.printSchema()

retaildata.registerTempTable("retaildata")



+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [30]:
from pyspark.sql.functions import desc,sum,count, col,year

In [4]:
retaildata.sort("InvoiceNo").explain()
retaildata.sort(desc("InvoiceNo")).explain()
retaildata.sort(desc("InvoiceNo")).limit(5).explain()

== Physical Plan ==
*(1) Sort [InvoiceNo#0 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(InvoiceNo#0 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#34]
   +- FileScan csv [InvoiceNo#0,StockCode#1,Description#2,Quantity#3,InvoiceDate#4,UnitPrice#5,CustomerID#6,Country#7] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/SQL and Explain Plan/2010-1..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:int,StockCode:string,Description:string,Quantity:int,InvoiceDate:timestamp,UnitP...


== Physical Plan ==
*(1) Sort [InvoiceNo#0 DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(InvoiceNo#0 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#46]
   +- FileScan csv [InvoiceNo#0,StockCode#1,Description#2,Quantity#3,InvoiceDate#4,UnitPrice#5,CustomerID#6,Country#7] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/E

In [5]:
retaildata.limit(5).explain()

== Physical Plan ==
CollectLimit 5
+- FileScan csv [InvoiceNo#0,StockCode#1,Description#2,Quantity#3,InvoiceDate#4,UnitPrice#5,CustomerID#6,Country#7] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/SQL and Explain Plan/2010-1..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:int,StockCode:string,Description:string,Quantity:int,InvoiceDate:timestamp,UnitP...




In [6]:
retaildata.groupBy("InvoiceNo").sum("Quantity").explain()

== Physical Plan ==
*(2) HashAggregate(keys=[InvoiceNo#0], functions=[sum(cast(Quantity#3 as bigint))])
+- Exchange hashpartitioning(InvoiceNo#0, 200), ENSURE_REQUIREMENTS, [id=#78]
   +- *(1) HashAggregate(keys=[InvoiceNo#0], functions=[partial_sum(cast(Quantity#3 as bigint))])
      +- FileScan csv [InvoiceNo#0,Quantity#3] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/SQL and Explain Plan/2010-1..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:int,Quantity:int>




In [9]:
retaildata.groupBy("InvoiceNo").agg(sum("Quantity")).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[InvoiceNo#0], functions=[sum(cast(Quantity#3 as bigint))])
+- Exchange hashpartitioning(InvoiceNo#0, 200), ENSURE_REQUIREMENTS, [id=#97]
   +- *(1) HashAggregate(keys=[InvoiceNo#0], functions=[partial_sum(cast(Quantity#3 as bigint))])
      +- FileScan csv [InvoiceNo#0,Quantity#3] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/SQL and Explain Plan/2010-1..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:int,Quantity:int>




In [10]:
retaildata.groupBy("InvoiceNo").agg(sum("Quantity")).withColumnRenamed("sum(miles)", "destination_total").explain()

== Physical Plan ==
*(2) HashAggregate(keys=[InvoiceNo#0], functions=[sum(cast(Quantity#3 as bigint))])
+- Exchange hashpartitioning(InvoiceNo#0, 200), ENSURE_REQUIREMENTS, [id=#116]
   +- *(1) HashAggregate(keys=[InvoiceNo#0], functions=[partial_sum(cast(Quantity#3 as bigint))])
      +- FileScan csv [InvoiceNo#0,Quantity#3] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/SQL and Explain Plan/2010-1..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:int,Quantity:int>




In [13]:
retaildata.groupBy("InvoiceNo").agg(count("Quantity")).withColumnRenamed("sum(miles)", "destination_total").explain()

== Physical Plan ==
*(2) HashAggregate(keys=[InvoiceNo#0], functions=[count(Quantity#3)])
+- Exchange hashpartitioning(InvoiceNo#0, 200), ENSURE_REQUIREMENTS, [id=#135]
   +- *(1) HashAggregate(keys=[InvoiceNo#0], functions=[partial_count(Quantity#3)])
      +- FileScan csv [InvoiceNo#0,Quantity#3] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/SQL and Explain Plan/2010-1..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:int,Quantity:int>




In [14]:
retaildata.groupBy("InvoiceNo").agg(count("Quantity")).withColumnRenamed("sum(miles)", "destination_total").sort("InvoiceNo").explain()

== Physical Plan ==
*(3) Sort [InvoiceNo#0 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(InvoiceNo#0 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#165]
   +- *(2) HashAggregate(keys=[InvoiceNo#0], functions=[count(Quantity#3)])
      +- Exchange hashpartitioning(InvoiceNo#0, 200), ENSURE_REQUIREMENTS, [id=#161]
         +- *(1) HashAggregate(keys=[InvoiceNo#0], functions=[partial_count(Quantity#3)])
            +- FileScan csv [InvoiceNo#0,Quantity#3] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/SQL and Explain Plan/2010-1..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:int,Quantity:int>




In [20]:
retaildata.filter((col("Country").isNotNull()) & (col("Country") == "United States")).groupBy("InvoiceNo").sum("Quantity").sort("InvoiceNo").explain()

== Physical Plan ==
*(3) Sort [InvoiceNo#0 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(InvoiceNo#0 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#247]
   +- *(2) HashAggregate(keys=[InvoiceNo#0], functions=[sum(cast(Quantity#3 as bigint))])
      +- Exchange hashpartitioning(InvoiceNo#0, 200), ENSURE_REQUIREMENTS, [id=#243]
         +- *(1) HashAggregate(keys=[InvoiceNo#0], functions=[partial_sum(cast(Quantity#3 as bigint))])
            +- *(1) Project [InvoiceNo#0, Quantity#3]
               +- *(1) Filter (isnotnull(Country#7) AND (Country#7 = United States))
                  +- FileScan csv [InvoiceNo#0,Quantity#3,Country#7] Batched: false, DataFilters: [isnotnull(Country#7), (Country#7 = United States)], Format: CSV, Location: InMemoryFileIndex[file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/SQL and Explain Plan/2010-1..., PartitionFilters: [], PushedFilters: [IsNotNull(Country), EqualTo(Country,United States)], ReadSchema: struct<InvoiceNo:int,Quantity:int,Coun

In [24]:
orders = spark.read.option("inferSchema","true").option("header","true").csv("orders.csv")
orders.show()
order_items = spark.read.option("inferSchema","true").option("header","true").csv("order_items.csv")
order_items.show()

orders.registerTempTable("orders")
order_items.registerTempTable("order_items")

+--------+----------+------+-----------+
|order_id|order_date|amount|customer_id|
+--------+----------+------+-----------+
|       1|07/04/1776|234.56|          1|
|       2|03/14/1760|  78.5|          3|
|       3|05/23/1784| 124.0|          2|
|       4|09/03/1790|  65.5|          3|
|       5|07/21/1795|  25.5|          5|
|       6|11/27/1787|  14.4|          7|
+--------+----------+------+-----------+

+--------+------+---------+--------+---------+
|order_id|itemid|productid|quantity|unitprice|
+--------+------+---------+--------+---------+
|       2|   123|     null|      12|     null|
|       1|   456|     null|      34|     null|
|       3|   789|     null|      56|     null|
|       5|   123|     null|      12|     null|
|       6|   456|     null|      34|     null|
|       3|   789|     null|      12|     null|
|       4|   123|     null|      12|     null|
|       5|   456|     null|      56|     null|
|       2|   789|     null|      12|     null|
|       3|   123|     nul

In [27]:
spark.sql("""SELECT year(order_date) as year, sum(quantity * unitprice) as revenue FROM orders NATURAL JOIN order_items GROUP BY year(order_date) ORDER BY year""").explain()

== Physical Plan ==
*(4) Sort [year#474 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(year#474 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#507]
   +- *(3) HashAggregate(keys=[year(cast(order_date#390 as date))#479], functions=[sum((cast(quantity#437 as double) * cast(unitprice#438 as double)))])
      +- Exchange hashpartitioning(year(cast(order_date#390 as date))#479, 200), ENSURE_REQUIREMENTS, [id=#503]
         +- *(2) HashAggregate(keys=[year(cast(order_date#390 as date)) AS year(cast(order_date#390 as date))#479], functions=[partial_sum((cast(quantity#437 as double) * cast(unitprice#438 as double)))])
            +- *(2) Project [order_date#390, quantity#437, unitprice#438]
               +- *(2) BroadcastHashJoin [order_id#389], [order_id#434], Inner, BuildRight, false
                  :- *(2) Filter isnotnull(order_id#389)
                  :  +- FileScan csv [order_id#389,order_date#390] Batched: false, DataFilters: [isnotnull(order_id#389)], Format: CSV, Locati

In [45]:
spark.sql("""select year(order_date) AS year, sum(quantity*unitprice)
            from orders o INNER JOIN order_items i
            ON o.order_id = i.order_id 
            GROUP BY year(order_date) 
            ORDER BY year""").explain()

== Physical Plan ==
*(4) Sort [year#769 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(year#769 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#730]
   +- *(3) HashAggregate(keys=[year(cast(order_date#390 as date))#774], functions=[sum((cast(quantity#437 as double) * cast(unitprice#438 as double)))])
      +- Exchange hashpartitioning(year(cast(order_date#390 as date))#774, 200), ENSURE_REQUIREMENTS, [id=#726]
         +- *(2) HashAggregate(keys=[year(cast(order_date#390 as date)) AS year(cast(order_date#390 as date))#774], functions=[partial_sum((cast(quantity#437 as double) * cast(unitprice#438 as double)))])
            +- *(2) Project [order_date#390, quantity#437, unitprice#438]
               +- *(2) BroadcastHashJoin [order_id#389], [order_id#434], Inner, BuildRight, false
                  :- *(2) Filter isnotnull(order_id#389)
                  :  +- FileScan csv [order_id#389,order_date#390] Batched: false, DataFilters: [isnotnull(order_id#389)], Format: CSV, Locati

In [41]:
orders.join(order_items,orders["order_id"] == order_items["order_id"],"inner").groupBy(year(col("order_date")).alias("year")).agg(sum(col("quantity") * col("unitprice"))).sort("year").explain()

== Physical Plan ==
*(4) Sort [year#749 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(year#749 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#662]
   +- *(3) HashAggregate(keys=[year(cast(order_date#390 as date))#763], functions=[sum((cast(quantity#437 as double) * cast(unitprice#438 as double)))])
      +- Exchange hashpartitioning(year(cast(order_date#390 as date))#763, 200), ENSURE_REQUIREMENTS, [id=#658]
         +- *(2) HashAggregate(keys=[year(cast(order_date#390 as date)) AS year(cast(order_date#390 as date))#763], functions=[partial_sum((cast(quantity#437 as double) * cast(unitprice#438 as double)))])
            +- *(2) Project [order_date#390, quantity#437, unitprice#438]
               +- *(2) BroadcastHashJoin [order_id#389], [order_id#434], Inner, BuildRight, false
                  :- *(2) Filter isnotnull(order_id#389)
                  :  +- FileScan csv [order_id#389,order_date#390] Batched: false, DataFilters: [isnotnull(order_id#389)], Format: CSV, Locati